# __Modelos pre entrenados de Tensorflow Hub__

### Ajustar los datos en funcion del modelo a utilizar

Vamos a trabajar con las imagenes de que tenemos precargadas en local.

In [13]:
# librerias
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import string
import tensorflow as tf

from tensorflow.keras import regularizers

# libreria para trabajar con las bases de datos 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Creamos nuevos data generators, donde modificamos el target size y el color mode. Esto será aplicado a los 3 generadores.

In [14]:
# ruta de los archivos
train_dir = './sign-language-img/Train'
test_dir = './sign-language-img/Test'

# data generator
train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255, validation_split=0.2)

# generator para test, training y validation adaptados al modelo
train_generator_resize = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    batch_size = 128,
    class_mode = 'categorical',
    color_mode = 'rgb',
    subset = 'training'
)

# para datos de validacion
validation_generator_resize = test_datagen.flow_from_directory(
    test_dir,
    target_size = (150, 150),
    batch_size = 128,
    class_mode = 'categorical',
    color_mode = 'rgb',
    subset = 'validation'
)

# para datos de prueba
test_generator_resize = test_datagen.flow_from_directory(
    test_dir,
    target_size = (150, 150),
    batch_size = 128,
    class_mode = 'categorical',
    color_mode = 'rgb'
)

Found 27455 images belonging to 24 classes.
Found 1425 images belonging to 24 classes.
Found 7172 images belonging to 24 classes.


Definimos las clases de los datos

In [15]:
# clases
classes = [char for char in string.ascii_uppercase if char != 'J' if char != 'Z']

### Cargar modelo de Hub a utilizar 

Importamos la libreria

In [16]:
import tensorflow_hub as hub

Antes de todo, debemos detectar qué configuración usaremos, para esta ocasión usaremos la arquitectura [MobileNetV1](https://www.kaggle.com/models/google/mobilenet-v1/tensorFlow2/050-160-classification/1?tfhub-redirect=true).

Este modelo es secuencial, por lo que stackearemos sus capas en un modelo de este tipo.

Para los modelos del hub, basta con agregar una capa de entrada, añadir la capa de KerasLayer con la URL del sitio (no olvides configurarlo como no entrenable) y a partir de este punto podrás inyectar tu propia arquitectura.

In [17]:
model_url = 'https://tfhub.dev/google/imagenet/mobilenet_v1_050_160/classification/4'

model_hub = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(150, 150, 3)),
    hub.KerasLayer(model_url, trainable=False),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(len(classes), activation='softmax')
])

ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x7efcdc38d7f0> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)